# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770345907670                   -0.53    8.0         
  2   -2.771685589993       -2.87       -1.31    1.0    194ms
  3   -2.771714310263       -4.54       -2.59    1.0    132ms
  4   -2.771714683502       -6.43       -3.79    1.0    162ms
  5   -2.771714714082       -7.51       -4.01    2.0    212ms
  6   -2.771714715222       -8.94       -5.33    1.0    140ms
  7   -2.771714715248      -10.58       -5.52    2.0    159ms
  8   -2.771714715250      -11.90       -6.16    2.0    175ms
  9   -2.771714715250      -13.70       -6.82    1.0    155ms
 10   -2.771714715250      -14.35       -7.46    1.0    165ms
 11   -2.771714715250      -13.97       -8.03    1.0    147ms


-0.00013457451635037128

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770549074889                   -0.53    9.0         
  2   -2.771780582326       -2.91       -1.32    1.0    138ms
  3   -2.771801794199       -4.67       -2.49    1.0    186ms
  4   -2.771802019108       -6.65       -3.23    1.0    196ms
  5   -2.771802073495       -7.26       -3.96    2.0    157ms
  6   -2.771802074107       -9.21       -4.20    1.0    174ms
  7   -2.771802074459       -9.45       -4.93    1.0    143ms
  8   -2.771802074471      -10.92       -5.22    1.0    148ms
  9   -2.771802074476      -11.33       -5.87    2.0    196ms
 10   -2.771802074476      -12.73       -6.38    1.0    149ms
 11   -2.771802074476      -14.10       -6.56    1.0    172ms
 12   -2.771802074476   +  -14.65       -6.91    1.0    152ms
 13   -2.771802074476      -13.81       -7.44    2.0    197ms
 14   -2.771802074476      -14.88       -7.55    1.0    156ms
 15   -2.

0.017612221288703992

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457451635037128
Displaced dipole:  0.017612221288703992
Polarizability :   1.7746795805054363


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920618532e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551411011e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852773128907e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593761844e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787393915e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.273414174841e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 6.491961688187e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 6.491961688187e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.191159280301e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.335425346527e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.747000837484e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.025207134776e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6.025207134776e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.